In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import math
from ast import literal_eval

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Range1d, LinearColorMapper, ColorBar, BasicTicker, CustomJS, DateSlider
from bokeh.layouts import layout, column, row
from bokeh.palettes import Spectral3, Viridis, Inferno256, Viridis256
from bokeh.tile_providers import CARTODBPOSITRON, STAMEN_TERRAIN

output_notebook()


Loading BokehJS ...

In [2]:
# Link data set file:
# https://www.dropbox.com/s/o889qz7gpzf0cdn/2015%20USA%20Weather%20Data%20FINAL.csv?dl=0
df = pd.read_csv('2015 USA Weather Data FINAL.csv',sep=';')
df = df.drop_duplicates(subset='STATION_NAME', keep='first')
# remove row that have '0' statename
df = df[(df.StateName != '0')]
df = df.drop(['STATION', 'LATLONG', 'Zip', 'State'], axis = 1)
# df

In [3]:
def merc(Coords):
    Coordinates = literal_eval(Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [4]:
df['Date'] = df['Date'].apply(lambda x: x.split()[0])

df['Location'] = df[['LATITUDE', 'LONGITUDE']].apply(lambda x : '({},{})'.format(x[0],x[1]), axis=1)
df['coords_x'] = df['Location'].apply(lambda x: merc(x)[0])
df['coords_y'] = df['Location'].apply(lambda x: merc(x)[1])

In [5]:
# print(source.data['index'][2])
df.head()

,STATION_NAME,LATITUDE,LONGITUDE,AvgTemp,MaxTemp,MinTemp,StateName,Date,Location,coords_x,coords_y
0,WORCESTER MA US,42.2706,-71.8731,55.0,58.0,47.0,Massachusetts,10/1/15,"(42.2706,-71.8731)",-8.000877e+06,5.201601e+06
31,CHICKEN ALASKA AK US,64.0600,-141.9294,27.0,39.0,17.0,Alaska,10/1/15,"(64.06,-141.9294)",-1.579951e+07,9.365017e+06
62,CHITNA ALASKA AK US,61.5319,-144.4397,34.0,45.0,24.0,Alaska,10/1/15,"(61.5319,-144.4397)",-1.607895e+07,8.748991e+06
93,INDIAN WELL CALIFORNIA CA US,41.7417,-121.5383,53.0,56.0,50.0,California,10/1/15,"(41.7417,-121.5383)",-1.352958e+07,5.122366e+06
124,CHATANIKA ALASKA AK US,65.0167,-148.5833,27.0,36.0,20.0,Alaska,10/1/15,"(65.0167,-148.5833)",-1.654022e+07,9.612772e+06


In [6]:
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

# range bounds supplied in web mercator coordinates
p = figure(x_range=(df['coords_x'].min(), df['coords_x'].max()), y_range=(df['coords_y'].min(), df['coords_y'].max()),
           x_axis_type="mercator", y_axis_type="mercator",
          tools=TOOLS )
p.add_tile(CARTODBPOSITRON)

COLORS = Inferno256
# Viridis256
# Inferno256
N_COLORS = len(COLORS)

color_low = df['MinTemp'].min() - 5
color_high = df['MaxTemp'].max() + 5

# color_low = -50
# color_high = 100

def color_value(AvgTemp):
    return COLORS[int((AvgTemp - color_low) / (color_high - color_low) * N_COLORS)]
    
df['color'] = df['AvgTemp'].apply(lambda x: color_value(x))
colors = [x for x in df['color']]
# print(colors)

color_mapper = LinearColorMapper(palette="Inferno256", low=color_low, high=color_high)

# if color.value != 'None':
#     if len(set(df[color.value])) > N_SIZES:
#         groups = pd.qcut(df[color.value].values, N_COLORS, duplicates='drop')
#     else:
#         groups = pd.Categorical(df[color.value])
#     c = [COLORS[xx] for xx in groups.codes]
# print(c)

color_bar = ColorBar(color_mapper=color_mapper, border_line_color=None, location=(0,0))
# df

In [7]:
# กำหนด source
true_source = ColumnDataSource(df)
source = ColumnDataSource(df.head())

In [8]:
p.add_layout(color_bar, 'right')

p.circle(x = source.data['coords_x'], y = source.data['coords_y'], fill_color = source.data['color'], line_color=source.data['color'], fill_alpha=0.4, size=5)

GlyphRenderer(id='1079', ...)

In [9]:
callback = CustomJS(args=dict(source=source, true_source=true_source), code="""
function timeConverter(UNIX_timestamp){
  var a = new Date(UNIX_timestamp);
  var months = ['01','02','03','04','05','06','07','08','09','10','11','12'];
  var year = a.getFullYear();
  var month = months[a.getMonth()];
  var date = a.getDate();
  var time = month + '/' + date + '/' + year;
  return time;
}

var f = cb_obj.value;

var data_ts = true_source.data;
var data_n=source.data;

var f=cb_obj.value;
var selected_date = timeConverter(f);

var coords_x=[];
var coords_y=[];
var color=[];

var date_s = new Date(selected_date);
var date_ts = new Date();

for(var i = 0; i < data_ts.Date.length; i++){
    console.log(i);
    date_ts = new Date(data_ts.Date[i]);
    console.log(date_s == date_ts);
    if(date_ts == date_s){
        coords_x.push(data['coords_x'][i]);
        coords_y.push(data['coords_y'][i]);
        color.push(data['color'][i]);
    }
}
data_n['coords_x']= coords_x;
data_n['coords_y']= coords_y;
data_n['color']= color;

source.change.emit();
""")

In [10]:
n = 1
def update(attr, old, new):
    n = n+1
    print(n)

In [11]:
date_start = "4/10/15"
date_end = "11/11/16"
date_init = "1/1/16"

date_slider = DateSlider(start=date_start, end=date_end, value=date_init, step=1, title="Date")
# date_slider.js_on_change('value', callback)
date_slider.on_change('value', update)

In [12]:
layout = column(p, date_slider)
show(layout, notebook_handle=True)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html

